In [2]:
# Import necessary packages
import pm4py
import pandas as pd
import json
import os
import shutil

from augm_baseline.ml.core.model import AugmentationExperiment
from augm_baseline.ml.core import model
from augm_baseline.ml.pipeline.augmentation_pipeline import run_pipeline

# Generate baseline data

In this notebook we apply the augmentation appraoch *Model-Agnostic Event Log Augmentation for Predictive Process Monitoring* from *M. Käppel* and *S. Jablonski* with the source code available at: https://github.com/mkaep/pbpm-ssl-suite

In [3]:
# Generate log file for training data
for name in ['helpdesk', 'sepsis', 'bpic13c', 'bpic15_1']:

    # Load data
    train_orig = pd.read_csv(f'data\\{name}\\{name}_orig\\train_{name}_orig.csv')
    train_orig['time:timestamp'] = pd.to_datetime(train_orig['time:timestamp'])
    train_orig[['case:concept:name', 'concept:name']] = train_orig[['case:concept:name', 'concept:name']].astype(str)

    # Convert it to an event log and save it
    train_xes = pm4py.convert_to_event_log(train_orig)
    pm4py.write_xes(train_xes, f'data\\{name}\\{name}_orig\\train_{name}_orig_80.xes')

c:\Users\Win10\anaconda3\envs\pbpm-experiment\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
exporting log, completed traces :: 100%|██████████| 767/767 [00:02<00:00, 293.39it/s]


After generating the event log files we can now load the defined experiments and create the augmented training data based on this baseline approach.

In [10]:
for name in ['helpdesk', 'sepsis', 'bpic13c', 'bpic15_1']:

    # Load experiment from baseline folder
    with open(f'augm_baseline\\experiments\\aug_{name}.json', 'r') as file:
        exp_data = json.load(file)

    # Generate experiment
    experiment = AugmentationExperiment.from_dict(exp_data)

    # Run the augmentation pipeline
    run_pipeline(experiment, verbose=True)

    # Load augmented data
    df = pd.read_csv(f'augm_baseline\\data\\{name}\\3_mixed_True_1.6\\train_3_mixed_True_1.6_augm.csv')
    df = df[['case:concept:name', 'concept:name', 'time:timestamp']]
    
    # Adjust case IDs as in some approaches we need numerical case IDs
    df['case:concept:name'] = df['case:concept:name'].str.replace('_', '', regex = False)

    # Save baseline training data
    os.makedirs(f'data\\{name}\{name}_baseline', exist_ok=True)
    df.to_csv(f'data\\{name}\\{name}_baseline\\train_{name}_baseline.csv', index=False)

    # Load original validation and test data
    df_val = pd.read_csv(f'data\\{name}\\{name}_orig\\val_{name}_orig.csv')
    df_test = pd.read_csv(f'data\\{name}\\{name}_orig\\test_{name}_orig.csv')

    # Generate full dataset with augmented training data
    df_total = pd.concat([df, df_val, df_test]).reset_index(drop=True)

    # Save full dataset
    total_path = f'data\\{name}\\{name}_baseline\\{name}_baseline.csv'
    df_total.to_csv(total_path, index=False)

    # Move original validation and test data to new folder
    for type in ['val', 'test']:
        source = f'data\\{name}\\{name}_orig\\{type}_{name}_orig.csv'
        dest = f'data\\{name}\\{name}_baseline\\{type}_{name}_baseline.csv'

        shutil.copy2(src=source, dst=dest)

 Start creating folder for 1 event logs and 0 approaches
 Create directory for event log helpdesk
 Load event log from data\helpdesk\helpdesk_orig\train_helpdesk_orig_80.xes


parsing log, completed traces :: 100%|██████████| 2931/2931 [00:01<00:00, 2655.35it/s]


Start augmentation...
BEFORE FIT:
[RandomInsertion, Activities: []]
[RandomDeletion]
[ParallelSwap]
[FragmentAugmentation]
[ReworkActivity]
[DeleteReworkActivity]
[RandomReplacement, Activities: []]
[RandomSwap]
[LoopAugmentation, max_additional_repetitions: 3, duration_tolerance: 0.2]
AFTER FIT:
[RandomInsertion, Activities: ['Resolve SW anomaly', 'Insert ticket', 'Resolve ticket', 'Closed', 'Create SW anomaly', 'INVALID', 'RESOLVED', 'Wait', 'VERIFIED', 'Schedule intervention', 'Assign seriousness', 'Take in charge ticket']]
[RandomDeletion]
[ParallelSwap]
[FragmentAugmentation]
[ReworkActivity]
[DeleteReworkActivity]
[RandomReplacement, Activities: ['Resolve SW anomaly', 'Insert ticket', 'Resolve ticket', 'Closed', 'Create SW anomaly', 'INVALID', 'RESOLVED', 'Wait', 'VERIFIED', 'Schedule intervention', 'Assign seriousness', 'Take in charge ticket']]
[RandomSwap]
[LoopAugmentation, max_additional_repetitions: 3, duration_tolerance: 0.2]
Starting augmentation
Augmentor could not appli

parsing log, completed traces :: 100%|██████████| 672/672 [00:01<00:00, 546.12it/s]


Start augmentation...
BEFORE FIT:
[RandomInsertion, Activities: []]
[RandomDeletion]
[ParallelSwap]
[FragmentAugmentation]
[ReworkActivity]
[DeleteReworkActivity]
[RandomReplacement, Activities: []]
[RandomSwap]
[LoopAugmentation, max_additional_repetitions: 3, duration_tolerance: 0.2]
AFTER FIT:
[RandomInsertion, Activities: ['Release A', 'CRP', 'ER Sepsis Triage', 'IV Antibiotics', 'ER Registration', 'Release D', 'Return ER', 'Release E', 'Release C', 'Admission IC', 'LacticAcid', 'Admission NC', 'Release B', 'ER Triage', 'IV Liquid', 'Leucocytes']]
[RandomDeletion]
[ParallelSwap]
[FragmentAugmentation]
[ReworkActivity]
[DeleteReworkActivity]
[RandomReplacement, Activities: ['Release A', 'CRP', 'ER Sepsis Triage', 'IV Antibiotics', 'ER Registration', 'Release D', 'Return ER', 'Release E', 'Release C', 'Admission IC', 'LacticAcid', 'Admission NC', 'Release B', 'ER Triage', 'IV Liquid', 'Leucocytes']]
[RandomSwap]
[LoopAugmentation, max_additional_repetitions: 3, duration_tolerance: 0.

parsing log, completed traces :: 100%|██████████| 951/951 [00:00<00:00, 1610.02it/s]


Start augmentation...
BEFORE FIT:
[RandomInsertion, Activities: []]
[RandomDeletion]
[ParallelSwap]
[FragmentAugmentation]
[ReworkActivity]
[DeleteReworkActivity]
[RandomReplacement, Activities: []]
[RandomSwap]
[LoopAugmentation, max_additional_repetitions: 3, duration_tolerance: 0.2]
AFTER FIT:
[RandomInsertion, Activities: ['Unmatched', 'Accepted', 'Completed', 'Queued']]
[RandomDeletion]
[ParallelSwap]
[FragmentAugmentation]
[ReworkActivity]
[DeleteReworkActivity]
[RandomReplacement, Activities: ['Unmatched', 'Accepted', 'Completed', 'Queued']]
[RandomSwap]
[LoopAugmentation, max_additional_repetitions: 3, duration_tolerance: 0.2]
Starting augmentation
Augmentor could not applied to this trace. We try to augment an other trace or with an other augmentor
Augmentor could not applied to this trace. We try to augment an other trace or with an other augmentor
Augmentor could not applied to this trace. We try to augment an other trace or with an other augmentor
Augmentor could not applie

parsing log, completed traces :: 100%|██████████| 767/767 [00:03<00:00, 239.89it/s]


Start augmentation...
BEFORE FIT:
[RandomInsertion, Activities: []]
[RandomDeletion]
[ParallelSwap]
[FragmentAugmentation]
[ReworkActivity]
[DeleteReworkActivity]
[RandomReplacement, Activities: []]
[RandomSwap]
[LoopAugmentation, max_additional_repetitions: 3, duration_tolerance: 0.2]
AFTER FIT:
[RandomInsertion, Activities: ['01_BB_765', '01_HOOFD_440_1', '01_HOOFD_490_3', '10_UOV_170', '08_AWB45_090_4', '01_BB_755', '01_BB_550_1', '07_OPS_080_1', '02_DRZ_010', '11_AH_II_080_1', '01_HOOFD_495', '08_AWB45_070_3', '01_HOOFD_350_2', '12_AP_UOV_020_2', '01_HOOFD_490_5a', '01_HOOFD_451', '01_HOOFD_140', '16_LGSV_010', '04_BPT_040', '14_VRIJ_011', '03_GBH_005', '08_AWB45_005', '08_AWB45_080', '01_BB_546', '08_AWB45_070_1', '10_UOV_060', '08_AWB45_090_1', '01_HOOFD_814', '07_OPS_010', '02_DRZ_010_1', '08_AWB45_150', '01_HOOFD_020', '01_BB_670_2', '09_AH_I_040', '06_VD_030_1a', '01_HOOFD_580_1', '08_AWB45_070_5', '06_VD_030_1', '01_HOOFD_090', '01_HOOFD_160', '06_VD_060_4', '11_AH_II_040c', 